# Loading the libraries

In [34]:
import csv
import pandas as pd
import os
from shutil import copyfile
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dense,Conv2D,MaxPool2D, Flatten
from tensorflow.keras.optimizers import Adam

# Creating Expression Image Dictionary

In [11]:
data={}
with open("data\\facial_expressions\\data\\legend.csv") as f:
    reader=csv.reader(f)
    next(reader)
    for row in reader:
        key=row[2].lower()  
        if key in data:
            data[key].append(row[1])
        else:
            data[key]=[row[1]]
    


# Creating and Populating Directories for ImageDataGenerator Class

### Total number of categories

In [12]:
data.keys()

dict_keys(['anger', 'surprise', 'disgust', 'fear', 'neutral', 'happiness', 'sadness', 'contempt'])

### Making the directories

In [13]:
os.mkdir('master_data')
os.mkdir('master_data/training')
os.mkdir('master_data/testing')

In [16]:
for emotion in data.keys():
    os.mkdir(os.path.join('master_data/training',emotion))
    os.mkdir(os.path.join('master_data/testing', emotion))
    

# Copying the files

In [26]:
split_size=0.8

for emotion, images in data.items():
    train_size= int(split_size*len(images))
    train_images=images[:train_size]
    test_images=images[train_size:]
    
    for image in train_images:
        source= os.path.join('data\\facial_expressions\\images',image)
        #print(source)
        destination=os.path.join('master_data\\training',emotion,image)
        #print(destination)
        copyfile(source,destination)
    
    for image in test_images:
        source= os.path.join('data\\facial_expressions\\images',image)
        #print(source)
        destination=os.path.join('master_data\\testing',emotion,image)
        #print(destination)
        copyfile(source,destination)        


# Creating the model

In [41]:
model=tf.keras.models.Sequential([
    Conv2D(16, (3,3), activation='relu', input_shape=(100,100,3)),
    MaxPool2D(2,2),
    Conv2D(32, (3,3), activation='relu'),
    MaxPool2D(2,2),
    Conv2D(64, (3,3), activation='relu'),
    MaxPool2D(2,2),
    Flatten(),
    Dense(1024, activation='relu'),
    Dense(8, activation='softmax')
    
])

In [42]:
model.compile(optimizer=Adam(learning_rate=0.01),loss='categorical_crossentropy',metrics=['accuracy'])

In [43]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 98, 98, 16)        448       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 49, 49, 16)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 47, 47, 32)        4640      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 23, 23, 32)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 21, 21, 64)        18496     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 10, 10, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 6400)             